# Finding, downloading, and processing relevant X-ray observations

We want to examine the X-ray emission of these groups, some of which we know have had targeted XMM observations. The XGA module doesn't support telescopes other than XMM at the moment, but we may download other available observations for further context/external analysis.

Once identified, the observations will be downloaded, and their raw data processed into a state suitable for analysis. This is a general search for all observations that may be relevant to our groups, they will not all necessarily be used for analysis.

## Import statements

In [1]:
import daxa
daxa.NUM_CORES = 20
daxa.OUTPUT = "/mnt/ufs18/rs-033/mc2/OVI_groups/data/"
daxa.config.OUTPUT = "/mnt/ufs18/rs-033/mc2/OVI_groups/data/"
daxa.mission.base.OUTPUT = "/mnt/ufs18/rs-033/mc2/OVI_groups/data/"
from daxa.archive import Archive
from daxa.process.simple import full_process_xmm
from daxa.mission import XMMPointed, Chandra, NuSTARPointed, ROSATPointed, Swift, Suzaku, ASCA
from daxa.exceptions import NoObsAfterFilterError

import pandas as pd
from astropy.units import Quantity
import os
from warnings import warn

## Reading the sample file

Reading in the sample file that contains the positions of the groups, these will be used to locate relevant X-ray observations:

In [2]:
samp = pd.read_csv("../sample_files/opt_group_info.csv")
samp

,name,opt_ra,opt_ra_err,opt_dec,opt_dec_err,redshift,ov_obs_ad,r_obs_ad,quasar,OVI_det
0,25124,243.538,0.015,26.687,0.006,0.186,448.0,1167.0,B1612+266,True
1,44739,229.822,0.019,28.342,0.058,0.118,336.0,1135.0,FBQS1519+2838,True
2,19670,150.178,0.028,50.784,0.007,0.134,404.0,991.0,SBS0956+509,False
3,12833,129.400,0.013,44.282,0.010,0.145,444.0,1396.0,RBS711,False
4,J083829.2+441904.8,NaN,NaN,NaN,NaN,0.198,NaN,NaN,RBS711,False
5,44858,230.084,0.035,28.873,0.008,0.127,569.0,1546.0,FBQS1519+2838,True
6,J215504.8-093520,NaN,NaN,NaN,NaN,0.077,NaN,NaN,PHL1811,True
7,J215437.2-091532.4,NaN,NaN,NaN,NaN,0.077,NaN,NaN,PHL1811,True
8,50433,157.732,0.003,30.845,0.006,0.136,338.0,600.0,FBQSJ1030+3102,False


Some groups in this table have NaN entries for optical positions, which is because we identified groups where a priori no X-ray emission was known. The group J083829.2+441904.8 was identified in the same XMM observation as 12833 (thus is removed from the table), and J215504.8-093520/J215437.2-091532.4 are in the foreground of quasar PHL 1811 (I know that this is slightly circular, but PHL 1811 was selected differently to all the other groups). For those we insert the coordinates of the quasar to search for X-ray observations

In [3]:
samp.loc[samp['name'].isin(['J215504.8-093520', 'J215437.2-091532.4']), ['opt_ra', 'opt_dec']] = [328.756, -9.373]
samp = samp[samp['name'] != "J083829.2+441904.8"].reset_index(drop=True)
samp

,name,opt_ra,opt_ra_err,opt_dec,opt_dec_err,redshift,ov_obs_ad,r_obs_ad,quasar,OVI_det
0,25124,243.538,0.015,26.687,0.006,0.186,448.0,1167.0,B1612+266,True
1,44739,229.822,0.019,28.342,0.058,0.118,336.0,1135.0,FBQS1519+2838,True
2,19670,150.178,0.028,50.784,0.007,0.134,404.0,991.0,SBS0956+509,False
3,12833,129.400,0.013,44.282,0.010,0.145,444.0,1396.0,RBS711,False
4,44858,230.084,0.035,28.873,0.008,0.127,569.0,1546.0,FBQS1519+2838,True
5,J215504.8-093520,328.756,NaN,-9.373,NaN,0.077,NaN,NaN,PHL1811,True
6,J215437.2-091532.4,328.756,NaN,-9.373,NaN,0.077,NaN,NaN,PHL1811,True
7,50433,157.732,0.003,30.845,0.006,0.136,338.0,600.0,FBQSJ1030+3102,False


## Searching for data

We can use DAXA classes to search the online databases of observations maintained for several X-ray telescopes. We will only make use of XMM data in this analysis, but it will be interesting to see what other data are available.

### XMM-Newton Pointed

We define an XMMPointed instance, which makes the distinction between XMM data that were taken in 'pointing' mode, when the attitude of the spacecraft was fixed (or nearly fixed), and data taken when XMM was slewing to its next target. Slew data is harder to analyse, with decreased spatial resolution, and is not yet implemented in DAXA.

The RA-Dec values for the groups of interest are used to filter the entire archive on position, searching for any XMM pointed observations with an aimpoint within the approximate field-of-view of the telescope:

In [40]:
# Define the instance
xmm = XMMPointed()
# This uses the pre-defined XMM field of view to search for observations around 
#  the positions of our groups - the same search will be performed using other telescopes
xm_filt_info = xmm.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
xm_filt_info['name'] = samp.loc[xm_filt_info['pos_ind'].astype(int), 'name'].values
xmm.filtered_obs_info

,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
917,157.746300,31.048890,0102040301,2000-12-06 20:46:26,True,0 days 08:08:31,2002-04-06 00:00:00,182,True,2000-12-07 04:54:57
922,157.746300,31.048890,0102041001,2000-12-07 04:57:14,True,0 days 01:30:06,2002-04-06 00:00:00,182,True,2000-12-07 06:27:20
3802,328.756200,-9.373528,0204310101,2004-11-01 09:06:42,True,0 days 09:08:39,2005-12-01 00:00:00,897,True,2004-11-01 18:15:21
12006,328.756250,-9.373333,0761910201,2015-11-29 09:38:07,True,0 days 16:30:00,2016-12-11 23:00:00,2925,True,2015-11-30 02:08:07
13847,230.111667,28.886111,0820240301,2019-01-14 04:17:53,True,0 days 06:06:40,2020-02-04 23:00:00,3498,True,2019-01-14 10:24:33
14961,129.414208,44.284083,0861080201,2020-10-07 12:28:01,True,0 days 06:18:20,2021-11-13 00:00:00,3815,True,2020-10-07 18:46:21
14962,129.414208,44.284083,0861080501,2020-10-07 10:09:04,True,0 days 02:18:57,2021-11-13 00:00:00,3815,True,2020-10-07 12:28:01
14965,150.166375,50.783694,0861080101,2020-10-11 11:57:49,True,0 days 06:23:20,2021-11-02 00:00:00,3817,True,2020-10-11 18:21:09
14972,150.166375,50.783694,0861080601,2020-10-11 09:53:44,True,0 days 02:04:05,2021-11-02 00:00:00,3817,True,2020-10-11 11:57:49
16245,229.803958,28.380667,0900700201,2022-08-17 12:04:49,True,0 days 08:13:20,2023-08-30 00:00:00,4155,True,2022-08-17 20:18:09


This return from the filtering method shows us which ObsIDs have been associated with which group positions:

In [39]:
xm_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,0,243.538,26.687,0900700101,25124
1,1,229.822,28.342,0900700201,44739
2,2,150.178,50.784,"0861080101,0861080601",19670
3,3,129.4,44.282,"0861080201,0861080501",12833
4,4,230.084,28.873,0820240301,44858
5,5,328.756,-9.373,"0204310101,0761910201",J215504.8-093520
6,6,328.756,-9.373,"0204310101,0761910201",J215437.2-091532.4
7,7,157.732,30.845,"0102040301,0102041001",50433


### Chandra

We perform a similar search in the Chandra archive - be aware that the odd geometry of some of Chandra's detectors makes the search radius defined in the DAXA class quite approximate:

In [41]:
chandra = Chandra()
ch_filt_info = chandra.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
ch_filt_info['name'] = samp.loc[ch_filt_info['pos_ind'].astype(int), 'name'].values
chandra.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:804: UserWarning: Chandra FoV are difficult to define, as they can be strongly dependant on observation mode; as such take these as very approximate.
  fov = self.fov


,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,instrument,grating,data_mode
9069,129.49708,44.24889,15378,True,True,2013-01-04 02:14:34.000002,2013-01-04 07:49:14.000002,0 days 05:34:40,2014-01-04,GCL,ACIS-I,NONE,TE_00458
9685,130.03917,44.36500,21564,True,True,2019-01-04 18:55:19.999998,2019-01-05 00:24:59.999998,0 days 05:29:40,2020-01-07,GCL,ACIS-I,NONE,TE_006E6
12569,130.03917,44.36500,22035,True,True,2019-01-06 19:19:28.000001,2019-01-06 22:40:48.000001,0 days 03:21:20,2020-01-07,GCL,ACIS-I,NONE,TE_006E6
14633,328.75625,-9.37353,2958,True,True,2001-12-17 22:59:37.999999,2001-12-18 01:45:37.999999,0 days 02:46:00,2002-12-21,AGN,ACIS-S,NONE,TE_002A2
15072,328.75625,-9.37353,2957,True,True,2001-12-05 08:43:41.000002,2001-12-05 11:22:01.000002,0 days 02:38:20,2002-12-18,AGN,ACIS-S,NONE,TE_002A2
18601,230.48917,28.98831,4791,True,True,2004-04-12 03:22:08.000002,2004-04-12 04:39:28.000002,0 days 01:17:20,2005-04-20,AGN,ACIS-S,NONE,TE_003C2
20635,328.75625,-9.37342,15357,True,True,2012-11-24 20:01:44.999999,2012-11-24 20:36:34.999999,0 days 00:34:50,2013-11-26,AGN,ACIS-S,NONE,TE_00B4A


In [42]:
ch_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,3,129.4,44.282,"15378,21564,22035",12833
1,4,230.084,28.873,4791,44858
2,5,328.756,-9.373,"2958,2957,15357",J215504.8-093520
3,6,328.756,-9.373,"2958,2957,15357",J215437.2-091532.4


We then download the Chandra data in their standard format, which will have pre-generated images that can be examined for emission from the groups we are interested in:

In [8]:
chandra.download(download_standard=True)

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/chandra.py:614: UserWarning: The raw data for this mission have already been downloaded.
  warn("The raw data for this mission have already been downloaded.")


### NuSTAR Pointed

We search the NuSTAR archive, though the small field-of-view of NuSTAR means that we do not find any serindipitious data (a try-except is used so that it doesn't halt the notebook):

In [44]:
nustar = NuSTARPointed()
nu_filt_info = nustar.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
nu_filt_info['name'] = samp.loc[nu_filt_info['pos_ind'].astype(int), 'name'].values
nustar.filtered_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,exposure_a,exposure_b,ontime_a,ontime_b,nupsdout,issue_flag
879,328.7406,-9.3989,60101004002,True,True,2015-11-28 18:46:08.184003,2015-11-30 04:11:08.184002,1 days 09:24:59.999999,2016-12-11 00:00:00,AGN,0 days 15:12:38,0 days 15:10:13,0 days 16:23:15,0 days 16:23:54,0,0


In [45]:
nu_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,5,328.756,-9.373,60101004002,J215504.8-093520
1,6,328.756,-9.373,60101004002,J215437.2-091532.4


In [29]:
nustar.download(download_processed=True)

## ROSAT Data

We can also search the ROSAT archive, though just the pointed observations, using DAXA missions. We do not necessarily expect to be able to see our groups in these data, but we might as well look.

The pointed phase could provide data from either the PSPC instrument (this type of instrument was used for the all-sky survey, and has a very large field of view), or the HRI (much higher spatial resolution, but essentially non-existent energy resolution). 

The search method uses appropriate field-of-view values for the different instruments to search for data:

In [46]:
rp = ROSATPointed()
rp_filt_info = rp.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
rp_filt_info['name'] = samp.loc[rp_filt_info['pos_ind'].astype(int), 'name'].values
rp.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,instrument,with_filter,target_category,target_name,proc_rev,fits_type
1402,157.75,31.05,RH800762N00,True,1996-05-11 12:00:40.999997,1996-05-27 17:24:16.000001,0 days 05:12:54,HRI,N,GCL,B2 1028+313,2,RDF 3_4
2657,150.06,50.62,RH701869N00,True,1995-05-17 23:58:44.999996,1995-05-19 00:43:29.999997,0 days 02:50:32,HRI,N,AGN,RX J10002+5036,2,RDF 3_4
2771,158.75,30.68,RP800350N00,True,1993-05-28 12:25:17.999996,1993-06-02 22:01:13.999996,0 days 02:43:51,PSPCB,N,GCL,A1045,2,RDF 3_4
3671,157.75,31.05,RF700433N00,True,1992-05-02 14:17:05.999997,1992-05-02 19:33:46.999999,0 days 01:54:48,PSPCB,Y,AGN,B2 1028+313,2,RDF 3_4
6954,157.75,31.05,RP700433N00,True,1992-05-01 19:08:26.999998,1992-05-02 19:01:36.000002,0 days 00:40:42,PSPCB,N,AGN,B2 1028+313,2,RDF 3_4


In [47]:
rp_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,2,150.178,50.784,RH701869N00,19670
1,7,157.732,30.845,"RP800350N00,RF700433N00,RP700433N00,RH800762N00",50433


Again we download with `download_products=True`, though HRI observations do not have exposure maps in the archive to be downloaded, we are not sure why:

In [12]:
rp.download(download_products=True)

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/rosat.py:602: UserWarning: The raw data for this mission have already been downloaded.
  warn("The raw data for this mission have already been downloaded.")


## ASCA Data

ASCA is an older mission, but as with ROSAT it may still be able to provide useful information. The field-of-view was respectable, though the sensitivity was fairly low:

In [48]:
asc = ASCA()
as_filt_info = asc.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
as_filt_info['name'] = samp.loc[as_filt_info['pos_ind'].astype(int), 'name'].values
asc.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/asca.py:87: UserWarning: 5 of the 3079 observations located for ASCA have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:55: UserWarning: There are multiple chosen instruments SIS0, SIS1, GIS2, GIS3 for asca with different FoVs, but they observe simultaneously. As such the search distance has been set to the largest FoV of the chosen instruments.
  any_ret = change_func(*args, **kwargs)


,ra,dec,ObsID,science_usable,start,end,duration,target_category,sis_exposure,gis_exposure
1388,157.656,31.0184,83063010,True,1995-05-07 03:03:27.360,1995-05-07 20:20:41.280,0 days 17:17:13.920000,GCL,0 days 07:10:56,0 days 07:52:32
2039,157.654,31.0191,83063000,True,1995-05-03 19:13:00.480,1995-05-04 05:40:24.960,0 days 10:27:24.480000,GCL,0 days 04:22:24,0 days 04:57:36


In [49]:
as_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,7,157.732,30.845,"83063010,83063000",50433


In [14]:
asc.download(download_processed=True)

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/asca.py:484: UserWarning: The raw data for this mission have already been downloaded.
  warn("The raw data for this mission have already been downloaded.")


## Suzaku Data

Suzaku was the successor to ASCA, and though the spectroscopy instrument failed, the CCD imager may have data relevant to our groups:

In [50]:
su = Suzaku()
su_filt_info = su.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
su_filt_info['name'] = samp.loc[su_filt_info['pos_ind'].astype(int), 'name'].values
su.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/suzaku.py:92: UserWarning: 14 of the 3055 observations located for Suzaku have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()


,ra,dec,ObsID,science_usable,start,end,duration,target_category,xis0_expo,xis0_num_modes,xis1_expo,xis1_num_modes,xis2_expo,xis2_num_modes,xis3_expo,xis3_num_modes
2125,243.6269,26.7212,807056010,True,2012-08-05 07:14:36.000004,2012-08-05 13:20:18.000001,0 days 06:05:41.999997,GCL,0 days 03:07:15,1,0 days 03:06:59.400000,1,0 days,0,0 days 03:07:15,1
2717,129.4975,44.2566,807061010,True,2012-10-13 19:51:53.999997,2012-10-14 08:26:13.000004,0 days 12:34:19.000007,GCL,0 days 05:43:41.900000,1,0 days 05:43:41.900000,1,0 days,0,0 days 05:43:41.900000,1


In [51]:
su_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,0,243.538,26.687,807056010,25124
1,3,129.4,44.282,807061010,12833


In [16]:
su.download(download_processed=True)

/tmp/local/32058688/ipykernel_117066/3261009431.py:1: UserWarning: The raw data for this mission have already been downloaded.
  su.download(download_processed=True)


## Swift Data

We also search Swift data for possible X-ray observations, and then proceed to select only one observation as the rest are unlikely to have long enough X-ray exposures to detect the groups. As with the other non-XMM and non-Chandra data, we may never use it, but it is acquired to provide additional information if needed:

In [52]:
sw = Swift(insts='xrt')
sw_filt_info = sw.filter_on_positions(samp[['opt_ra', 'opt_dec']].values, return_pos_obs_info=True)
sw_filt_info['name'] = samp.loc[sw_filt_info['pos_ind'].astype(int), 'name'].values
sw.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/swift.py:97: UserWarning: 546 of the 352048 observations located for Swift have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/swift.py:97: UserWarning: 5421 of the 352048 observations located for Swift have been removed due to all chosen instrument (XRT) exposures being zero.
  self._fetch_obs_info()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:55: UserWarning: There are multiple chosen instruments XRT for swift with different FoVs, but they observe simultaneously. As such the search distance has been set to the largest FoV of the chosen instruments.
  any_ret = change_func(*args, **kwargs)


,ra,dec,ObsID,science_usable,start,end,duration,target_category,xrt_exposure,bat_exposure,uvot_exposure
1230,157.76970,31.07815,00035373001,True,2006-07-01 08:33:59.999996,2006-07-03 14:00:07.999998,2 days 05:26:08.000002,MISC,0 days 04:42:31.607000,0 days 04:47:22,0 days 04:41:47.906000
18370,157.75703,31.06702,00035373004,True,2009-11-21 05:17:59.999997,2009-11-21 23:53:16.000002,0 days 18:35:16.000005,MISC,0 days 01:17:51.855000,0 days 01:19:32,0 days 01:15:15.487000
37862,229.80744,28.23228,00034974003,True,2017-04-12 10:02:57.000002,2017-04-13 00:00:48.999998,0 days 13:57:51.999996,MISC,0 days 00:46:11.911000,0 days 00:46:28,0 days 00:44:57.020000
42542,328.76223,-9.35815,00030335002,True,2005-10-22 09:49:01.999997,2005-10-22 11:51:36.999996,0 days 02:02:34.999999,MISC,0 days 00:41:36.221000,0 days 00:42:11,0 days 00:41:13.802000
66616,129.22863,44.45192,00087091010,True,2018-05-24 13:44:56.999996,2018-05-25 12:58:50.000002,0 days 23:13:53.000006,MISC,0 days 00:29:42.673000,0 days 00:30:01,0 days 00:28:52.841000
67834,229.83481,28.23837,00034974002,True,2017-03-10 23:50:57.000002,2017-03-11 03:57:03.000004,0 days 04:06:06.000002,MISC,0 days 00:29:11.434000,0 days 00:29:29,0 days 00:27:57.601000
76542,229.81373,28.28268,00095208001,True,2019-11-17 01:43:35.999996,2019-11-17 06:00:23.000000,0 days 04:16:47.000004,MISC,0 days 00:26:28.400000,0 days 00:00:00,0 days 00:25:39.689000
76918,328.75738,-9.39494,00030335003,True,2006-05-12 00:49:59.999998,2006-05-12 02:20:14.000004,0 days 01:30:14.000006,MISC,0 days 00:26:21.776000,0 days 00:26:39,0 days 00:25:55.520000
85169,129.13529,44.41848,00087091003,True,2018-04-02 12:06:57.000001,2018-04-02 14:46:04.999996,0 days 02:39:07.999995,MISC,0 days 00:23:43.610000,0 days 00:24:57,0 days 00:23:55.086000
94590,129.15719,44.42458,00087091006,True,2018-05-03 10:54:57.000001,2018-05-03 16:36:54.000003,0 days 05:41:57.000002,MISC,0 days 00:20:59.381000,0 days 00:21:13,0 days 00:20:09.935000


In [53]:
sw_filt_info

,pos_ind,pos_ra,pos_dec,ObsIDs,name
0,1,229.822,28.342,"00034974003,00034974002,00095208001,0003497400...",44739
1,3,129.4,44.282,"00087091010,00087091003,00087091006,00087091009",12833
2,5,328.756,-9.373,"00030335002,00030335003",J215504.8-093520
3,6,328.756,-9.373,"00030335002,00030335003",J215437.2-091532.4
4,7,157.732,30.845,"00035373001,00035373004",50433


In [54]:
sw.filter_on_obs_ids(['00035373001', '00035373004'])
sw.download(download_processed=True)

## Creating an Archive

We'll set up a DAXA archive - which is be most useful when processing multi-mission data, but here will just contain XMM data, as we can't yet process any of the other telescopes with DAXA. The archive will be named, and the processed raw data will be stored in it, as well as logs of the processing steps.

It also automatically downloads the XMM data, if we haven't already triggered that from the mission class instance:

In [19]:
ovi_group_arch = Archive(xmm, 'OVIGroups', clobber=True)
ovi_group_arch.info()

/tmp/local/32058688/ipykernel_117066/4266497429.py:1: UserWarning: An archive called OVIGroups already existed, but as clobber=True it has been deleted and overwritten.
  ovi_group_arch = Archive(xmm, 'OVIGroups', clobber=True)



-----------------------------------------------------
Number of missions - 1
Total number of observations - 11
Beginning of earliest observation - 2000-12-06 20:46:26
End of latest observation - 2022-09-12 10:34:47

-- XMM-Newton Pointed --
   Internal DAXA name - xmm_pointed
   Chosen instruments - M1, M2, PN
   Number of observations - 11
   Fully Processed - False
-----------------------------------------------------



/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:101: UserWarning: The number of cores assigned to XMMPointed downloads has been capped at 10, this will minimise dropped connections.
  mission.download()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:101: UserWarning: The raw data for this mission have already been downloaded.
  mission.download()


## Processing XMM data

The different processing steps that need to be applied to XMM can be controlled separately, with an array of user-configurable options; but a full processing stack to go from raw data to cleaned event lists, images, and exposure maps is available. It uses default settings and should produce very useful data:

In [20]:
full_process_xmm(ovi_group_arch)

XMM-Newton Pointed - Generating calibration files: 100%|██████████| 11/11 [01:15<00:00,  6.90s/it]
XMM-Newton Pointed - Generating ODF summary files: 100%|██████████| 11/11 [02:52<00:00, 15.68s/it]
XMM-Newton Pointed - Assembling PN and PN-OOT event lists: 100%|██████████| 8/8 [21:37<00:00, 162.17s/it]
XMM-Newton Pointed - Assembling MOS event lists: 100%|██████████| 20/20 [02:40<00:00,  8.02s/it]
XMM-Newton Pointed - Finding PN/MOS soft-proton flares: 100%|██████████| 28/28 [01:31<00:00,  3.28s/it]
XMM-Newton Pointed - Generating cleaned PN/MOS event lists: 100%|██████████| 28/28 [00:14<00:00,  1.94it/s]
XMM-Newton Pointed - Generating final PN/MOS event lists: 100%|██████████| 24/24 [00:00<00:00, 34.59it/s]
Generating products of type(s) expmap: 100%|██████████| 24/24 [04:24<00:00, 11.04s/it]


## Checking the data

Here we use the logging facilities of DAXA archives to investigate whether all the data were processed fully and, if they weren't, which observations failed and at which steps. That way we can decide whether there is something extra that can be done for those observations, or there are no further steps to take.

### Overall processing success

The first step is to check the 'overall' success, which will only report as False if all data for an entire observation (i.e. PN, MOS1, and MOS2) have been marked as failing processing. We can see that the processing of two observations has failed outright:

In [21]:
ovi_group_arch.final_process_success

{'xmm_pointed': {'0102040301': True,
  '0102041001': False,
  '0204310101': True,
  '0761910201': True,
  '0820240301': True,
  '0861080201': True,
  '0861080501': False,
  '0861080101': True,
  '0861080601': False,
  '0900700201': True,
  '0900700101': True}}

### 0102041001 - Outright failure

We can examine the 'observation summary' for 0102041001, which is parsed from the SAS summary file generated by ODF ingest. While the instruments all took science observations, we can see that each of the three telescopes had their filter wheels in 'CalClosed' position:

In [22]:
ovi_group_arch.observation_summaries['xmm_pointed']['0102041001']

{'M1': {'active': True,
  'num_exp': 1,
  'exposures': {'S004': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimePartialW3',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'M2': {'active': True,
  'num_exp': 1,
  'exposures': {'S006': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimePartialW3',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'PN': {'active': True,
  'num_exp': 1,
  'exposures': {'S003': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeSmallWindow',
    'filter': 'CalClosed',
    'ccd_modes': {4: 'Imaging'}}}}}

### 0861080601 - Outright failure

We can examine the 'observation summary' for 0861080601, which is parsed from the SAS summary file generated by ODF ingest. While the instruments all took science observations, we can see that each of the three telescopes had their filter wheels in 'CalClosed' position. Given the similar ObsID of the other outright failure, it seems likely that it will be for the same reason:

In [23]:
ovi_group_arch.observation_summaries['xmm_pointed']['0861080601']

{'M1': {'active': True,
  'num_exp': 1,
  'exposures': {'S001': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     7: 'Imaging'}}}},
 'M2': {'active': True,
  'num_exp': 1,
  'exposures': {'S002': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'PN': {'active': True,
  'num_exp': 14,
  'exposures': {'S003': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging',
     8: 'Imaging',
     9: 'Imaging',
     10: 'Imaging',
     11: 'Imaging',
     12: 'Imaging'

### 0861080501 - Outright failure

This has failed outright for the same reason as 0861080601, all observations were taken with the CalClosed filter:

In [24]:
ovi_group_arch.observation_summaries['xmm_pointed']['0861080501']

{'M1': {'active': True,
  'num_exp': 1,
  'exposures': {'S001': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     7: 'Imaging'}}}},
 'M2': {'active': True,
  'num_exp': 1,
  'exposures': {'S002': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'PN': {'active': True,
  'num_exp': 14,
  'exposures': {'S003': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging',
     8: 'Imaging',
     9: 'Imaging',
     10: 'Imaging',
     11: 'Imaging',
     12: 'Imaging'

### Tracing ObsID-Instrument-SubExposure failures at different stages

We can also check the success flags for the different processing stages at an individual sub-exposure level, to see whether any of the observations had a partial failure in processing. The only problematic sub-exposure we find is '0801892301PNS003', the scheduled PN observation of 0801892301, which failed the `espfilt` (soft-proton filtering) stage and was not allowed to continue:

In [25]:
ovi_group_arch.process_success

{'xmm_pointed': {'cif_build': {'0761910201': True,
   '0102040301': True,
   '0102041001': True,
   '0204310101': True,
   '0900700101': True,
   '0900700201': True,
   '0861080501': True,
   '0820240301': True,
   '0861080101': True,
   '0861080201': True,
   '0861080601': True},
  'odf_ingest': {'0102041001': True,
   '0861080601': True,
   '0861080501': True,
   '0820240301': True,
   '0102040301': True,
   '0861080101': True,
   '0204310101': True,
   '0861080201': True,
   '0900700101': True,
   '0761910201': True,
   '0900700201': True},
  'epchain': {'0102040301PNS003': True,
   '0820240301PNS003': True,
   '0861080101PNS003': True,
   '0861080201PNS003': True,
   '0900700101PNS003': True,
   '0204310101PNS003': True,
   '0900700201PNS003': True,
   '0761910201PNS003': True},
  'emchain': {'0861080201M1S001': True,
   '0861080201M2S002': True,
   '0900700201M1U003': True,
   '0900700201M2U003': True,
   '0861080201M1U003': True,
   '0102040301M1S004': True,
   '0861080101M1S001'

## Assigning regions to XMM observations

Source regions are a key part of any analysis, but DAXA does not currently have the capability to apply source detection to observations that it processes. As such we shall use source regions generated by the XMM Cluster Survey's (XCS) XAPA source finder. Passing them into the DAXA archive will store them in the DAXA archive directory structure:

In [26]:
reg_path = "../sample_files/xcs_regions/{o_id}_sky_regions.reg"

In [27]:
av_reg = [f.split('_')[0] for f in os.listdir('../sample_files/xcs_regions/')]
reg_dec = {'xmm_pointed': {o: {'regions': reg_path.format(o_id=o)} for o in av_reg}}

ovi_group_arch.source_regions = reg_dec